# Unsupervised Anomaly Detection Brain-MRI

Jupyter notebook for running all the experiments from our [paper](https://arxiv.org/abs/2004.03271). 

Hyperparameters may have to be adjusted!

## Preperation

### Imports and installation of the required libraries


In [ ]:
# from google.colab import drive
# from google.colab import files
import os, glob
! pip install pynrrd
! pip install SimpleITK
! pip install bunch
! pip install nibabel
! pip install medpy
! pip install opencv-python

### Get Code
Clone Code from github.com

In [2]:
# ! git clone https://github.com/irfixq/Investigate_WAE_in_BrainMRI
# ! cd Investigate_WAE_in_BrainMRI/

### Google Drive mount
Mounting Google Drive to access datasets.

In [3]:
# drive.mount('gdrive')

Check Directory

In [4]:
os.getcwd()

'C:\\Users\\Irfixq\\Desktop\\P2\\Code 1\\Unsupervised_Anomaly_Detection_Brain_MRI-master'

### Tensorboard and tunneling
Install ngrok for tunneling 

In [5]:
if os.path.exists("ngrok-stable-linux-amd64.zip"):
    os.remove("ngrok-stable-linux-amd64.zip")

if os.path.exists("ngrok"):
    os.remove("ngrok")
  
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


Start tensorboard and forward port with ngrok

In [ ]:
LOG_DIR = 'logs/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

Extract ngrok url for accessing tensorboard

**Attention**: Sometimes it throws an error like this:
```
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
```
If this is the case the easiest way to solve this issue is to delete the ngrok*.zip and ngrok from the Google Drive folder and install them again.


In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!pip3 install tensorflow==1.15

## Training

### Imports

In [ ]:
# %tensorflow_version 1.x
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import json
import os
import tensorflow as tf
import cv2
from datetime import datetime
from utils.default_config_setup import get_config, get_options, get_datasets
from trainers.AE import AE
from trainers.VAE import VAE
from trainers.CE import CE
from trainers.ceVAE import ceVAE
from trainers.GMVAE import GMVAE
from trainers.fAnoGAN import fAnoGAN
from trainers.AnoVAEGAN import AnoVAEGAN
from trainers.WAE import WAEGAN
from models import autoencoder, variational_autoencoder, context_encoder_variational_autoencoder,gaussian_mixture_variational_autoencoder,fanogan,constrained_autoencoder,anovaegan, WAEGAN
from utils import Evaluation
from utils.default_config_setup import get_config, get_options, get_datasets, Dataset


Set paths to datasets and where to save checkpoints and evaluations.

In [ ]:
def get_CONFIG(timestamp=None):
  current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
  if timestamp:
    current_time=timestamp
  dataset_root = "C:/Users/Irfixq/Desktop/P2/Code 1/Unsupervised_Anomaly_Detection_Brain_MRI-master"
  save_dir = "C:/Users/Irfixq/Desktop/P2/Code 1/Unsupervised_Anomaly_Detection_Brain_MRI-master"
  CONFIG = {
    "BRAINWEBDIR": os.path.join(dataset_root, 'Brainweb'),
    "CHECKPOINTDIR": os.path.join(save_dir, 'checkpoints', current_time),
    "SAMPLEDIR": os.path.join(save_dir, 'sample_dir', current_time),
  }
  return CONFIG

### Manual Training


#### Baseline

**AE**

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=AE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = AE(tf.Session(), config, network=autoencoder.autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**VAE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=VAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = VAE(tf.Session(), config, network=variational_autoencoder.variational_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


#### ceVAE - Variations

Paper: [Context-encoding Variational Autoencoder for Unsupervised Anomaly Detection](https://arxiv.org/abs/1812.05941)



**CE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.Brainweb
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=CE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = CE(tf.Session(), config, network=autoencoder.autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**ceVAE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.Brainweb
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=ceVAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.use_gradient_based_restoration = 0.002

# Create an instance of the model and train it
model = ceVAE(tf.Session(), config, network=context_encoder_variational_autoencoder.context_encoder_variational_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**VAE-Zimmerer**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=64, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=VAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = VAE(tf.Session(), config, network=variational_autoencoder_Zimmerer.variational_autoencoder_Zimmerer)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**ceVAE-Zimmerer**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.Brainweb
options = get_options(batchsize=64, learningrate=0.0001, numEpochs=1, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=ceVAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = ceVAE(tf.Session(), config, network=context_encoder_variational_autoencoder_Zimmerer.context_encoder_variational_autoencoder_Zimmerer)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


#### GMVAE-(Restoration)-Variations

Paper: [Unsupervised Lesion Detection via Image Restoration with a Normative Prior](https://openreview.net/forum?id=S1xg4W-leV)


**VAE-You**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=VAE_You, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.restore_lr = 1e-3
config.restore_steps = 10
config.tv_lambda = 0.0

# Create an instance of the model and train it
model = VAE_You(tf.Session(), config, network=variational_autoencoder.variational_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**GMVAE-You**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=GMVAE_spatial, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.dim_c = 9
config.dim_z = 1
config.dim_w = 1
config.c_lambda = 1
config.restore_lr = 1e-3
config.restore_steps = 10
config.tv_lambda = 1

# Create an instance of the model and train it
model = GMVAE_spatial(tf.Session(), config, network=gaussian_mixture_variational_autoencoder_You.gaussian_mixture_variational_autoencoder_You)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**GMVAE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=GMVAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.dim_c = 9
config.dim_z = 128
config.dim_w = 1
config.c_lambda = 1
config.restore_lr = 1e-3
config.restore_steps = 10
config.tv_lambda = 0.0

# Create an instance of the model and train it
model = GMVAE(tf.Session(), config, network=gaussian_mixture_variational_autoencoder.gaussian_mixture_variational_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))

**GMVAE-spatial**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=GMVAE_spatial, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.dim_c = 9
config.dim_z = 1
config.dim_w = 1
config.c_lambda = 1
config.restore_lr = 1e-3
config.restore_steps = 10
config.tv_lambda = 0.0

# Create an instance of the model and train it
model = GMVAE_spatial(tf.Session(), config, network=gaussian_mixture_variational_autoencoder_spatial.gaussian_mixture_variational_autoencoder_spatial)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))

#### f-AnoGAN

Paper: [f-AnoGAN: Fast unsupervised anomaly detection with generative adversarial networks.](https://www.ncbi.nlm.nih.gov/pubmed/30831356)

**Unified f-AnoGan**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=fAnoGAN, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.kappa = 1.0
config.scale = 10.0

# Create an instance of the model and train it
model = fAnoGAN(tf.Session(), config, network=fanogan.fanogan)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))

**f-AnoGAN - Schlegl**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=8, learningrate=0.0001, numEpochs=2, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=fAnoGAN, options=options, optimizer='ADAM', intermediateResolutions=[16, 16], dropout_rate=0.1, dataset=datasetHC)

config.kappa = 1.0
config.scale = 10.0

# Create an instance of the model and train it
model = fAnoGAN(tf.Session(), config, network=fanogan_schlegl.fanogan_schlegl)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


#### Constrained Adversarial AE

Paper: [Unsupervised Detection of Lesions in Brain MRI using constrained adversarial auto-encoders](https://arxiv.org/abs/1806.04972)

**constrained AAE**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=ConstrainedAAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.scale = 10.0
config.rho = 1.0

# Create an instance of the model and train it
model = ConstrainedAAE(tf.Session(), config, network=constrained_adversarial_autoencoder.constrained_adversarial_autoencoder)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))


**constrained AAE Chen**

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=8, learningrate=0.0001, numEpochs=2, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=ConstrainedAAE, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

config.kappa = 1.0
config.scale = 10.0
config.rho = 1.0

# Create an instance of the model and train it
model = ConstrainedAAE(tf.Session(), config, network=constrained_adversarial_autoencoder_Chen.constrained_adversarial_autoencoder_Chen)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))

#### AnoVAEGAN

Paper: [Deep autoencoding models for unsupervised anomaly segmentation in brain MR images](https://arxiv.org/abs/1804.04488)

In [ ]:
tf.reset_default_graph()
dataset = Dataset.BRAINWEB
options = get_options(batchsize=128, learningrate=0.0001, numEpochs=20, zDim=128, outputWidth=128, outputHeight=128, config=get_CONFIG())
options['data']['dir'] = options["globals"][dataset.value]
datasetHC, datasetPC = get_datasets(options, dataset)
config = get_config(trainer=AnoVAEGAN, options=options, optimizer='ADAM', intermediateResolutions=[8, 8], dropout_rate=0.1, dataset=datasetHC)

# Create an instance of the model and train it
model = AnoVAEGAN(tf.Session(), config, network=anovaegan.anovaegan)

# Train it
model.train(datasetHC)

# Evaluate
Evaluation.evaluate(datasetPC, model, options, description=f"{type(datasetHC).__name__}-{options['threshold']}", epoch=str(options['train']['numEpochs']))
